In [ ]:
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_datasets as tfds

We load the datset and split it into test and training

In [ ]:
#alternativ data_test, data_train = tfds.load('Malaria', as_supervised = True, split = ["train[:10%]", "train[10%:100%]"])
data_test, info_test = tfds.load('Malaria', as_supervised = True, split = "train[:10%]", with_info=True)
data_train = tfds.load('Malaria',as_supervised = True, split = "train[10%:100%]")
#data has shape (<tf.Tensor: shape=(103, 103, 3), dtype=uint8, numpy= array
# if not as_Supervised {'image': <tf.Tensor: shape=(142, 151, 3), dtype=uint8, numpy=array

In [ ]:
#split the dataset into training and test data

# Num classes: 2
print("Num classes: " + str(info_test.features['label'].num_classes))
#Class names: ['parasitized', 'uninfected']
print("Class names: " + str(info_test.features['label'].names))

vis = tfds.visualization.show_examples(data_test, info_test)

# gc.collect()  = garbage collector to regain capacities

for image, label in data_train.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

**Preprocessing**

Our images have different sizes but we don't want an accidental correlation between image size and label. Also, it is more convenient to have all data in the same shape when feeding it to the network. Therefore we use padding to make all images the same size, by adding some surrounding pixels.

In [ ]:
# these are the functions we use to pad

#def convert(image, label):
#  image = tf.image.convert_image_dtype(image, tf.float32)
#  return image, label

#Crops and/or pads an image to a target width and height
def pad(image,label):
  image,label = convert(image, label)
  image = tf.image.resize_with_crop_or_pad(image, 200, 200)
  return image,label

In [ ]:
# here we apply our pad function by using the map function
# while were at it we also bath

padded_data_train = (
    data_train
    .cache()
    .map(pad)
    .batch(128) # we use minibatches instead of sibgle samples to be more efficient
) 

padded_data_test = (
    data_test
    .cache()
    .map(pad)
    .batch(128)
) 

In [ ]:
vis = tfds.visualization.show_examples(padded_data_train, info_test) #thisonly works if you comment out the batching above
#here we find out the shape of the image, so that we can use it tp pass it to ur model
# (Image shape:  (200, 200, 3))
for image, label in padded_data_train.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())
    print(label)
    label = tf.one_hot(label, depth=2)
    print(label)

**Model**

Lets build a model for our convolutional neural network.
We need convolutional and pooling layers


In [ ]:
def conv_block(filters):
    block = tf.keras.Sequential([
        tf.keras.layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
        tf.keras.layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D()
    ]
    )
    
    return block

def dense_block(units, dropout_rate):
    block = tf.keras.Sequential([
        tf.keras.layers.Dense(units, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout_rate)
    ])
    
    return block

In [ ]:
#functions from opencast
# tf.keras.layers.MaxPool2D(
#    pool_size=(2, 2), strides=None, padding='valid', data_format=None, **kwargs
#)

#tf.keras.layers.GlobalAveragePooling2D(
#    data_format=None, **kwargs
#)

from tensorflow.keras.layers import Layer

# You will implement a CNN. The layers you need are: tf.keras.layers.Conv2D, tf.keras.layers.MaxPool2D, 
# tf.keras.layers.Flatten, tf.keras.layers.Dense

# Input
# Convolutional layer: 32 kernels of size (3,3) with ReLU activation function (tf.keras.activations.relu)
# Note: In the first layer you have to provide the input shape to the function (argument: input_shape).
# Maxpooling layer: pooling size (2,2) and strides (2,2).
# Convolutional layer: 64 kernels of size (3,3) with ReLU activation function 
# Maxpooling layer: pooling size (2,2) and strides (2,2).
# Convolutional layer: 64 kernels of size (3,3) with ReLU activation function
# Flatten the resulting feature maps.
# Fully connected layer with 64 hidden neurons and ReLU activation function.
# Fully connected layer with 10 output neurons and softmax activation function (tf.keras.activations.softmax).

# INitialize model
class Model(Layer):
    
    def __init__(self):
        super(Model, self).__init__()
        # Initialize all layers.
        # we alternate between convolutional and pooling layers
        #because this is the first convolutional layer we have to pass the input shape to it
        self.conv_layer_1 = tf.keras.layers.Conv2D(
                                filters=32,
                                kernel_size=3,
                                activation=tf.keras.activations.relu,
                                #input shape = shape of images plus extra dimension because of batching
                                input_shape=(200,200,3,1) 
                            )
        # our pooling layer, we use max-pooling
        self.max_pool_1 = tf.keras.layers.MaxPool2D()
        self.conv_layer_2 = tf.keras.layers.Conv2D(
                                filters=64,
                                kernel_size=3,
                                activation=tf.keras.activations.relu
                            )
        self.max_pool_2 = tf.keras.layers.MaxPool2D()
        self.conv_layer_3 = tf.keras.layers.Conv2D(
                                filters=64,
                                kernel_size=3,
                                activation=tf.keras.activations.relu
                            )
        self.flatten = tf.keras.layers.Flatten()
        self.fully_connected_1 = tf.keras.layers.Dense(
                                units=64,
                                activation=tf.keras.activations.relu
                            )
        # for the last layer we use a fully connected layer with sotmax activation function. Outputs are between 0 and 1 and add up to 1
        self.fully_connected_2 = tf.keras.layers.Dense(
                                units=10,
                                activation=tf.keras.activations.softmax
                            )
    
    # here the model gets called and the input gets passed through all the layers  
    def call(self, x):
        # Define the model.
        x = self.conv_layer_1(x)
        x = self.max_pool_1(x)
        x = self.conv_layer_2(x)
        x = self.max_pool_2(x)
        x = self.conv_layer_3(x)
        x = self.flatten(x)
        x = self.fully_connected_1(x)
        x = self.fully_connected_2(x)
        return x
        ######################

Train Model

In [ ]:
tf.keras.backend.clear_session()

# Initialize model, loss (binary cross entropy) and optimizer (Adam).
model = Model()
# we use the binary corss entropy to computethe loss because we have two labels (infected, uninfected)
binary_entropy_loss = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Initialize lists for later visualiztion
train_steps = []
train_losses = []
train_accuracies = []
test_steps = []
test_losses = []
test_accuracies = []
step = 0


for epoch in range(3):
    #t = target
    for (x,t) in padded_data_train:
        
        t = tf.reshape(t, shape=[-1])
        
        ### YOUR CODE HERE ###
        # Turn the labels into one-hot vectors.
        t = tf.one_hot(t, depth=10) #why depth 10? can we make it work with depth =2 siince we have 2 labels?
        
        # Perform a training step.
        with tf.GradientTape() as tape:
            output = model(x)
            loss = binary_entropy_loss(t, output)
            gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))  
        
        # Calculate the training accuracy every 25 steps.
        if step % 25 == 0:
            accuracy = np.sum(np.argmax(t, axis=1) == np.argmax(output, axis=1)) / t.shape[0]
            train_accuracies.append(accuracy)
            train_losses.append(loss)
            train_steps.append(step)
        
        # Calculate the test loss and accuracy every 50 steps.
        if step % 50 == 0:
            for (x,t) in padded_data_test:
                t = tf.reshape(t, shape=[-1])
                t = tf.one_hot(t, depth=10)
                output = model(x)
                loss = binary_entropy_loss(t, output)
                accuracy = np.sum(np.argmax(t, axis=1) == np.argmax(output, axis=1)) / t.shape[0]
                test_steps.append(step)
                test_accuracies.append(accuracy)
                test_losses.append(loss)
        ########################
        
        step += 1

Visualization

In [ ]:
import matplotlib.pyplot as plt
#goal: 95% accuracy

# Visualize accuracy and loss for training and test data. 
#Plot training and test loss.
plt.figure()
line1, = plt.plot(train_steps, train_losses)
line2, = plt.plot(test_steps, test_losses)
plt.xlabel("Training steps")
plt.ylabel("Loss")
plt.legend((line1,line2),("training","test"))
plt.show()

# Plot training and test accuracy.
plt.figure()
line1, = plt.plot(train_steps, train_accuracies)
line2, = plt.plot(test_steps, test_accuracies)
plt.xlabel("Training steps")
plt.ylabel("Accuracy")
plt.legend((line1,line2),("training","test"))
plt.show()
#####################